In [ ]:
import porter
from TextRepresenter import PorterStemmer
from ParserCACM import ParserCACM
import pickle as pkl


class Index():
	def __init__(self, name, parser, textRepresenter, dir):
		self.name = name
		self.docFrom = {}
		self.parser = parser
		self.textRepresenter = textRepresenter
		self.index = {}
		self.index_inverse = {}
		self.dir = dir
		self.index_file = dir + 'index_file'
		self.index_file_inverse = dir + 'index_inverse_file'

	def indexation(self, filename):
		self.parser.initFile(filename)

		#Fichier d'index
		index_file = open(self.index_file, 'wb')
		index_inverse_file = open(self.index_file_inverse, 'wb')
		doc_file = open(filename , 'r')


		# Premiere passe
		doc = self.parser.nextDocument()
		doc_source_place = 0
		doc_place = 0
		stem_place = 0
		print ("STEP COMPUTE")
		while (doc != None):
			id = doc.identifier
			if int(id) > 100:
				break
			print("COMPUTE doc " + str(id))
			self.docFrom[id] = (filename, doc_source_place, len(doc.text))
			doc_source_place += len(doc.text)
			# Index normal, on "reserve" la place
			bow = self.textRepresenter.getTextRepresentation(doc.text)
			str_bow = self._dict_to_file(bow)
			byte_size = len(str_bow)
			self.index[id] = (doc_place, byte_size)
			doc_place += byte_size
			#Pour chaque stem, on le rajoute en tant que cle
			for i in bow.keys():
				if i not in self.index_inverse.keys():
					# Si il n'existe pas, on initialise sa position et sa longueur a 0
					self.index_inverse[i] = (0, 0)
				#On met a jour sa longueur, si il existe, on addition sa longueur
				self.index_inverse[i] = (0, self.index_inverse[i][1] + len(self._line_to_file(id, bow[i])))
			doc = self.parser.nextDocument()

		#A ce stade, les position sont a 0, on met a jour a partir de la long precedente
		for stem in self.index_inverse.keys():
			self.index_inverse[stem] = (stem_place, self.index_inverse[stem][1])
			stem_place += self.index_inverse[stem][1]


		print("STEP WRITE")
		# Seconde passe
		self.parser.initFile(filename)
		doc = self.parser.nextDocument()
		stem_place = {}
		while (doc != None):
			id = doc.identifier
			if int(id) > 100:
				break
			print("WRITE doc " + str(id))
			# Index normal
			bow = self.textRepresenter.getTextRepresentation(doc.text)
			#Pour chaque doc, on met l'offset a sa position et on ecrit les "stem | tf"
			index_file.seek(self.index[id][0])
			index_file.write(self._dict_to_file(bow))

			#Index inverse
			#On prend la position dans index_reverse...
			for stem in bow.keys():
				if stem not in stem_place.keys():
					stem_place[stem] = self.index_inverse[stem][0]
				#L'offset est place a la position du stem et on ecrit "id_doc | tf"
				index_inverse_file.seek(stem_place[stem])
				index_inverse_file.write(self._line_to_file(id, bow[stem]))
				#Pour le meme stem, la position est additionee par sa longueur
				stem_place[stem] = stem_place[stem] + len(self._line_to_file(id, bow[stem]))
			doc = self.parser.nextDocument()
        
		doc_file.flush()
		doc_file.close()
		index_file.flush()
		index_inverse_file.flush()
		index_file.close()
		index_inverse_file.close()

	def getTfsForDoc(self, id_doc):
		index_file = open(self.index_file , 'r')
		index_file.seek(self.index[id_doc][0])
		stem_tf = index_file.read(self.index[id_doc][1])
		index_file.flush()
		index_file.close()
		return stem_tf

	def getTfsForStem(self, stem):
		index_file_inverse = open(self.index_file_inverse, 'r')
		index_file_inverse.seek(self.index_inverse[stem][0])
		doc_tf = index_file_inverse.read(self.index_inverse[stem][1])
		index_file_inverse.flush()
		index_file_inverse.close()
		return doc_tf

	def getStrDoc(self, id_doc):
		f = open(self.docFrom[id_doc][0], 'r')
		f.seek(self.docFrom[id_doc][1])
		doc = f.read(self.docFrom[id_doc][2])
		f.flush()
		f.close()
		return doc


	def _dict_to_file(self, dict):
		return ''.join([self._line_to_file(i, dict[i]) for i in dict.keys()])

	def _line_to_file(self, i, v):
		return str(i) + '|' + str(v) + ' '


tr = PorterStemmer()
parser = ParserCACM()
index = Index("test", parser, tr, "C:\Users\Jeremy\Documents\RI\out_cacm\\")
index.indexation("C:\Users\Jeremy\Documents\RI\cacm\cacm.txt")
rep = index.getTfsForDoc('99')
text = index.getStrDoc('99')
stem = index.getTfsForStem('system')

In [41]:
import pickle as pkl
index_file=open('cacmtest_index','rb')
a = index_file.read()
score = {1:1,2:2,3:3}
with open('donnees','wrb') as file:
    mypickler = pkl.Pickler(file)
    mypickler.dump(score)
    mypickler.dump(a)

In [44]:
with open('donnees','rb') as file:
    mypickler = pkl.Unpickler(file)
    a = mypickler.load()
    b = mypickler.load()
    print a,b

{1: 1, 2: 2, 3: 3} A|1 Internat|1 Samelson|1 Preliminari|1 K|1 J|1 Perli|1 Algebraic|1 Report|1 Languag|1 Digit|1 Repeat|1 Comput|1 I|1 Root|1 Extraction|1 Sugai|1 Subtract|1 Depart|1 Matrix|1 Friedman|1 M|1 Program|1 Scheme|1 Techniqu|1 D|1 Glossari|1 Programm|1 Comput|1 Terminolog|1 Engineer|1 Squar|1 G|1 Approxim|1 Two|1 W|1 Wadei|1 Root|1 


In [15]:
mypickler


<pickle.Unpickler instance at 0x7f31144a7ab8>

In [38]:
index[0][0]
print index[1:300],len(index)

[['1', 13], ['10', 47], ['10-bit', 47], ['1023', 47], ['1957', 85], ['1957-II', 83], ['1957-III', 58], ['2', 10], ['2', 83], ['220', 35], ['256', 65], ['40', 95], ['4800', 62], ['5000', 62], ['64', 47], ['64', 47], ['650', 32], ['650', 35], ["650's", 68], ['650-Part', 49], ['650-Part', 68], ['704', 30], ['704', 95], ['709', 60], ['=', 47], ['A', 0], ['A', 10], ['A', 10], ['A', 13], ['A', 13], ['A', 15], ['A', 16], ['A', 19], ['A', 20], ['A', 24], ['A', 25], ['A', 32], ['A', 32], ['A', 34], ['A', 35], ['A', 37], ['A', 38], ['A', 39], ['A', 39], ['A', 41], ['A', 42], ['A', 45], ['A', 47], ['A', 47], ['A', 53], ['A', 57], ['A', 64], ['A', 65], ['A', 67], ['A', 68], ['A', 72], ['A', 73], ['A', 77], ['A', 77], ['A', 79], ['A', 81], ['A', 82], ['A', 86], ['A', 88], ['A', 90], ['A', 91], ['A', 91], ['A', 91], ['A', 92], ['A', 94], ['ALGOL', 37], ['ALGOL', 52], ['ALGOL', 63], ['ALGOL', 64], ['AN', 57], ['Accelerat', 19], ['Acton,', 37], ['Adams,', 17], ['Address', 78], ['Administr', 47], ['Aeg

In [ ]:
class Index(name,docs,stems,docFrom,parser,text_Representer):
    def indexation(self,data):
        self.
        return self
    def getTfsForDoc(self,document):
        return stem-tf
    def getTfsForStem(self,stem):
        return doc-tf
    def getStrDoc(self,)

In [ ]:
j=0
for i in x2_text_list:
    x2_text_list[j] = stem(i)
    j=j+1
print x2_text_list

In [ ]:
x3 = a.nextDocument()

In [ ]:
x4 = a.nextDocument()

In [ ]:
print x3

In [35]:
a ='ss'
print a,type(a) is str

ss True
